# Pipeline
1. Generate numbers
2. Extract features
3. Create classification model with linear classifier
4. Train with gradient descent (using auto_grad from pytorch)

## Generate numbers

In [7]:
import torch

def generate_data():
    # generage number matrix
    image_data=[]
    num_0 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,1,0,0,1,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_0)
    num_1 = torch.tensor(
    [[0,0,0,1,0,0],
    [0,0,1,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,1,1,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_1)
    num_2 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0],
    [0,1,1,1,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_2)
    num_3 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_3)
    num_4 = torch.tensor(
    [
    [0,0,0,0,1,0],
    [0,0,0,1,1,0],
    [0,0,1,0,1,0],
    [0,1,1,1,1,1],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_4)
    num_5 = torch.tensor(
    [
    [0,1,1,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,1,0,0],
    [0,0,0,0,1,0],
    [0,1,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_5)
    num_6 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_6)
    num_7 = torch.tensor(
    [
    [0,1,1,1,1,0],
    [0,0,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_7)
    num_8 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_8)
    num_9 = torch.tensor(
    [[0,0,1,1,1,0],
    [0,1,0,0,1,0],
    [0,0,1,1,1,0],
    [0,1,0,0,1,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_9)
    return image_data

## Feature extraction function
HOG

LBP

SUFR

SIFT

In [289]:
def hog(image):
    image = image.numpy()
    hog = cv2.HOGDescriptor()
    h, w = image.shape[:2]
    rate = 64 / w
    image = cv2.resize(image, (64, np.int(rate*h)))
    bg = np.zeros((128, 64), dtype=np.uint8)
    print(bg.shape,image.shape)
    bg[:,:] = 127
    h, w = image.shape
    dy = (128 - h) // 2
    bg[dy:h+dy,:] = image
    descriptors = hog.compute(bg, winStride=(8, 8), padding=(0, 0))
    return descriptors

def LBP(gray_image, window=3):
    lbp = np.zeros_like(gray_image)
    h,w = gray_image.shape
    p_center = window//2
    for ph in range(0,h-window):
        for pw in range(0,w-window):
            img = gray_image[ph:ph+window, pw:pw+window]
            center = img[p_center,p_center]
            binary = (img>=center)*1.0
            binary_vector = binary.flatten()
            #remove center point
            binary_vector = np.delete(binary_vector,4)
            res = 0
            for i in range(len(binary_vector)):
                if binary_vector[i]==1:
                    res+= 2**i
            lbp[ph+p_center,pw+p_center] = res
    return lbp

def get_feature(x):
    """feature extraction"""
    feature = torch.sum(x,0)+torch.sum(x,1)
    feature = feature
    return feature

image_data = generate_data()
print(get_feature(image_data[0]))

tensor([2, 5, 4, 4, 5, 0])


## Classification model

In [349]:
import random
import numpy as np
class model:
    def __init__(self, sample_input, lr=0.001,out_channels=10,feature = get_feature):
        """
        a binary classifier to classify target digit
        sample_input:used to automatically set shape of weight matrices
        lr: learning rate
        """
        self.channels=out_channels
        self.w = torch.tensor(np.random.rand(*([self.channels]+list(sample_input.shape))))
        self.lr = lr
        
        #built-in functions for forward and backward calculations
        self.feature = feature
    def forward(self,x):
        self.x = x
        #print(self.w.shape,x.view(-1,1).shape)
        x = torch.mm(self.w,x.view(-1,1).double()) #matrix multiplication
        out = x/sum(x) #normalize to become probability logits
        #built-in data storage for backward calculation
        self.y_hat = out.flatten()
        return out
    def loss(self,y,LossFunc = None):
        if isinstance(y,int): #if input is an integer
            y=[y]
        y = np.array(y)
        if len(y.shape)==1: #requires one-hot encoding of y
            one_hot = np.zeros((y.size, self.channels))
            one_hot[np.arange(y.size),y]=1
            y = one_hot.flatten()
        self.y = torch.tensor(y)
        return torch.mean((self.y_hat-self.y)**2)
    def update(self):
        gradient = 2*(self.y_hat-self.y).view(-1,1)*self.x.view(1,-1)
        self.w -= self.lr*gradient
        return
    def predict(self,x):
        logits = self.forward(x)
        return torch.argmax(logits)
        
        

m = model(get_feature(image_data[0]))
m.forward(get_feature(image_data[0]))
loss = m.loss(0)
m.update()
m.predict(get_feature(image_data[0]))

a = np.array([9])
b = np.zeros((a.size, 9+1))
b[np.arange(a.size),a] = 1
torch.argmax(torch.tensor(b))

a = get_feature(image_data[0])
w = torch.tensor(np.random.rand(*([8]+list(a.shape))))
print(a.shape,w.shape)
out = torch.mm(w,a.view(-1,1).double())
out.shape

## Train

In [676]:
def train_and_test(feature,epochs,mode='train',print_every = 100, model=model,**kwargs):
    labels = [0,1,2,3,4,5,6,7,8,9]
    m = model(feature(image_data[0]),0.1,out_channels=len(labels))
    for epoch in range(epochs):
        loss = 0
        for i,x in enumerate(image_data):
            v = feature(x)
            y = labels[i]
            o = m.forward(v)
            if mode=='debug':
                return m
            cur_loss = m.loss(y)
            loss+=cur_loss
            m.update()
            pred = m.predict(v)


        if epoch%print_every==0:
            print("epoch: {}, loss: {}".format(epoch,loss))
    for i,x in enumerate(image_data):
        v = feature(x)
        print('prediction of number {} is :{}'.format(i,m.predict(v)))
    return m


In [403]:
#this feature function does not distinguish between 6 and 8
feature = get_feature
epochs=1000
train_and_test(feature,epochs)

epoch: 0, loss: 1.034706817362325
epoch: 100, loss: 0.5874769553484933
epoch: 200, loss: 0.5636475721187735
epoch: 300, loss: 0.5475244920066803
epoch: 400, loss: 0.5347499267654419
epoch: 500, loss: 0.5246355669785214
epoch: 600, loss: 0.5166412079336254
epoch: 700, loss: 0.5103285784581413
epoch: 800, loss: 0.5053480904946634
epoch: 900, loss: 0.5014221551998018
prediction of number 0 is :0
prediction of number 1 is :1
prediction of number 2 is :2
prediction of number 3 is :3
prediction of number 4 is :4
prediction of number 5 is :6
prediction of number 6 is :6
prediction of number 7 is :7
prediction of number 8 is :6
prediction of number 9 is :9


In [358]:
#use flattened x as output directly, enforce overfitting in real case
feature = lambda x: x.flatten()
epochs=1000
train_and_test(feature,epochs)

epoch: 0, loss: 0.9386055958690203
epoch: 100, loss: 0.17530046658964013
epoch: 200, loss: 0.09131533697425613
epoch: 300, loss: 0.05952200558154673
epoch: 400, loss: 0.04165872882866667
epoch: 500, loss: 0.03012948033006923
epoch: 600, loss: 0.02221338059343832
epoch: 700, loss: 0.016577221351129084
epoch: 800, loss: 0.01246788258669094
epoch: 900, loss: 0.00942404622307222
prediction of number 0 is :0
prediction of number 1 is :1
prediction of number 2 is :2
prediction of number 3 is :3
prediction of number 4 is :4
prediction of number 5 is :5
prediction of number 6 is :6
prediction of number 7 is :7
prediction of number 8 is :8
prediction of number 9 is :9


In [419]:
#does not properly train

def get_feature2(x):
    """feature extraction from ex2"""
    img = x[:5,]
    lr = torch.flip(img,[1])
    lr_sym = img - lr
    lr_sum = sum(sum(abs(lr_sym)))
    ud = torch.flip(img,[0])
    ud_sym = img-ud
    ud_sum = sum(sum(abs(ud_sym)))
    mid_sum = sum(img[3,:])
    #added center element to distinguish between 0 and 8
    center = img[2,2]+1
    #convert tensor to float for more digit precision
    #convert back to int to prevent additional floating point calculation
    out =torch.cat((lr_sum.view(-1),ud_sum.view(-1),center.view(-1)))
    #print(out)
    return out

for image in image_data:
    print(get_feature2(image))
feature = get_feature2
epochs=1000
m = train_and_test(feature,epochs,lr = 0.001,mode='train')

tensor([0, 0, 1])
tensor([8, 6, 1])
tensor([ 4, 10,  1])
tensor([4, 0, 2])
tensor([14,  6,  2])
tensor([10,  4,  2])
tensor([4, 2, 2])
tensor([ 8, 10,  1])
tensor([0, 0, 2])
tensor([6, 4, 2])
epoch: 0, loss: 1.0746936978336885
epoch: 100, loss: 0.9868466251490494
epoch: 200, loss: 0.9868481050656508
epoch: 300, loss: 0.986848105076528
epoch: 400, loss: 0.986848105076528
epoch: 500, loss: 0.986848105076528
epoch: 600, loss: 0.986848105076528
epoch: 700, loss: 0.986848105076528
epoch: 800, loss: 0.986848105076528
epoch: 900, loss: 0.986848105076528
prediction of number 0 is :8
prediction of number 1 is :7
prediction of number 2 is :2
prediction of number 3 is :5
prediction of number 4 is :4
prediction of number 5 is :5
prediction of number 6 is :9
prediction of number 7 is :7
prediction of number 8 is :8
prediction of number 9 is :7


In [427]:
#This is not better than first method

def get_feature3(x):
    feature=[0,0,0,0]
    # 下面添加提取图像x的特征feature的代码
    def get_shadow(x,dim):
        feature  =torch.sum(x,dim)
        feature = feature.float()
        # 归一化
        for i in range(0,feature.shape[0]):
            feature[i]=feature[i]/sum(feature)

        feature = feature.view(1,6)
        return feature
    feature  = get_shadow(x,0)
    #import pdb
    #pdb.set_trace()
    #print(feature)
    return feature.flatten()

for image in image_data:
    print(get_feature3(image))
    
feature = get_feature3
epochs=10000
m = train_and_test(feature,epochs,lr = 0.1,mode='train')

tensor([0.0000, 0.3000, 0.2740, 0.3588, 0.7628, 0.0000])
tensor([0.0000, 0.0000, 0.2500, 0.8000, 0.4878, 0.0000])
tensor([0.0000, 0.2000, 0.3659, 0.5390, 0.6442, 0.0000])
tensor([0.0000, 0.0000, 0.3750, 0.5581, 0.6819, 0.0000])
tensor([0.0000, 0.0909, 0.1982, 0.2413, 0.7657, 0.4355])
tensor([0.0000, 0.3636, 0.4074, 0.6288, 0.4167, 0.0000])
tensor([0.0000, 0.3000, 0.4110, 0.6368, 0.4259, 0.0000])
tensor([0.0000, 0.1250, 0.1404, 0.6384, 0.6888, 0.0000])
tensor([0.0000, 0.2000, 0.3659, 0.5390, 0.6442, 0.0000])
tensor([0.0000, 0.1818, 0.2178, 0.2703, 0.8818, 0.0000])
epoch: 0, loss: 0.9311403056610096
epoch: 100, loss: 0.6902613454432747
epoch: 200, loss: 0.6176524563579978
epoch: 300, loss: 0.5813448952287088
epoch: 400, loss: 0.5601661088343235
epoch: 500, loss: 0.5467450786116319
epoch: 600, loss: 0.5377250915680553
epoch: 700, loss: 0.5314011745582023
epoch: 800, loss: 0.5268336226879181
epoch: 900, loss: 0.5234657504788891
epoch: 1000, loss: 0.5209466959750263
epoch: 1100, loss: 0.519

In [411]:
m.y_hat

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=torch.float64)

## Train with auto_grad

1. requires_grad: to retain gradient
    1. To check status of a variable: a.requires_grad
2. Cannot use backward with vector, only scalar. Because loss function is a scalar. Easy to adapt for vector
3. Cannot change graph of grad, two ways to update weights:
    1. with torch.nn.no_grad():
    2. update weights.data directly
4. Need to zero gradient after every update: 
    1. weights.grad.data.zero_()
    2. weights.grad.data = zeros(....)
5. Cannot backward() for more than once: RuntimeError. Need to use retain_grad (retain the calculated values) in order to backward multiple times. 
6. Leaf node:
    1. Used to save memory
    2. Leaf nodes have 'None" grad_fn
    3. None-leaf nodes have grad_fn that's not 'None'
    4. Determine if a variable is leaf node: a.is_leaf
    5. If we want to retain gradients of intermediate variable, use retain_grad. Memory will be release by default if not set this way.
    6. If we only need to output grad information of intermediate variables but not saving them, use tensor.register_hook
        1. e.g. loss2.register_hook(lambda grad:print("loss grad:",grad))
        2. memory is released after printing
7. In-place operation
    1. pytorch use b._version to detect. Every time a tensor has inplace operation, version increment by 1. 
    2. If not inplace operation, varible id increments. (check by id(a))
    3. in-place operation not allowed for leaf variables with requires_grad=True [RuntimeError: leaf variable has been moved into the graph interior]
    4. ways to change data of leaf variables:
        1. a.data.fill_ (or other built-in operations)
        2. with torch.no_grad():
    5. a.data shares memory with a, but with requires_grad = False

### Examples with auto_grad

In [471]:
x = torch.randn(1,10)
w = torch.ones(10,1,requires_grad=True)
w

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], requires_grad=True)

In [472]:
y = x.mm(w)
print(y)
loss = (y-8)*(y-8)
print(loss)

tensor([[-5.7640]], grad_fn=<MmBackward>)
tensor([[189.4480]], grad_fn=<MulBackward0>)


In [473]:
#gradience calculation
loss.backward()
g = w.grad
w = w +g

In [680]:
#model with auto_grad


    

class model_auto_grad:
    def __init__(self, sample_input, lr=0.001,out_channels=10,feature = get_feature):
        """
        a binary classifier to classify target digit
        sample_input:used to automatically set shape of weight matrices
        lr: learning rate
        target: the target digit to classify as IS TARGET or NOT TARGET
        """
        self.channels=out_channels
        self.w = torch.tensor(np.random.rand(*([self.channels]+list(sample_input.shape))),requires_grad=True)
        #self.b = torch.tensor(np.random.rand(self.channels,1),requires_grad=True)
        self.lr = lr
        #built-in functions for forward and backward calculations
        self.feature = feature
    def forward(self,x):
        self.x = x
        #print(self.w.shape,x.view(-1,1).shape)
        x = torch.mm(self.w,x.view(-1,1).double()) #matrix multiplication
        #x = x+self.b
        out = x/sum(x) #normalize to become probability logits
        #built-in data storage for backward calculation
        self.y_hat = out.flatten()
        return out
    def L2(self,y):
        return torch.mean((self.y_hat-self.y)**2)
    def CrossEntropy(self, y):
        loss = torch.tensor(0.0,requires_grad=True)
        i=0
        yHat = self.y_hat
        #print(yHat)
        for pred in yHat:
            if y[i] == 1:
                 loss = loss - torch.log(pred)
            else:
                loss = loss -torch.log(1 - pred)
            i+=1
        return loss/i
    def loss(self,y,LossFunc = CrossEntropy):
        if isinstance(y,int): #if input is an integer
            y=[y]
        y = np.array(y)
        if len(y.shape)==1: #requires one-hot encoding of y
            one_hot = np.zeros((y.size, self.channels))
            one_hot[np.arange(y.size),y]=1
            y = one_hot.flatten()
        self.y = torch.tensor(y)
        #use cross entropy here
        #CrossEntropy.requres_grad = True
        self.loss_val = LossFunc(self,y)
        #self.loss_val = torch.mean((self.y_hat-self.y)**2)
        self.loss_val.backward()
        return self.loss_val
        
    def update(self):
        #print(self.w.grad)
        g_w = self.w.grad.data
        #g_b = self.b.grad.data
        self.w.data -= self.lr*g_w
        #self.b.data -= self.lr*g_b
        self.w.grad.data.zero_()
        #self.b.grad.data.zero_()
        return
    def predict(self,x):
        logits = self.forward(x)
        return torch.argmax(logits)
        
        

In [677]:
#use L2 loss
feature = lambda x: x.flatten()
epochs=10000
m = train_and_test(feature,epochs,model=model_auto_grad,lr=1,print_every=1000)

epoch: 0, loss: 0.892553625233369
epoch: 1000, loss: 0.8091974416705341
epoch: 2000, loss: 0.342985607084156
epoch: 3000, loss: 0.047219599635461185
epoch: 4000, loss: 0.004394038533564421
epoch: 5000, loss: 0.0004145834866336078
epoch: 6000, loss: 4.2994079831998436e-05
epoch: 7000, loss: 4.6875652503879375e-06
epoch: 8000, loss: 5.279212613092977e-07
epoch: 9000, loss: 6.087098345455412e-08
prediction of number 0 is :0
prediction of number 1 is :1
prediction of number 2 is :2
prediction of number 3 is :3
prediction of number 4 is :4
prediction of number 5 is :5
prediction of number 6 is :6
prediction of number 7 is :7
prediction of number 8 is :8
prediction of number 9 is :9


In [ ]:
feature = lambda x: x.flatten()
epochs=1000
m = train_and_test(feature,epochs,model=model_auto_grad,lr=0.01)

In [679]:
#use L2 loss
feature = get_feature
epochs=10000
m = train_and_test(feature,epochs,model=model_auto_grad,lr=0.01,print_every=1000)

epoch: 0, loss: 0.9177928427830887
epoch: 1000, loss: 0.8818461794976101
epoch: 2000, loss: 0.7744264048204932
epoch: 3000, loss: 0.46891190756080103
epoch: 4000, loss: 0.47503682193944263
epoch: 5000, loss: 0.41032769604410213
epoch: 6000, loss: 0.41187691804068166
epoch: 7000, loss: 0.40818076416436433
epoch: 8000, loss: 0.4079299012959047
epoch: 9000, loss: 0.40757624078808863
prediction of number 0 is :0
prediction of number 1 is :1
prediction of number 2 is :2
prediction of number 3 is :3
prediction of number 4 is :4
prediction of number 5 is :6
prediction of number 6 is :6
prediction of number 7 is :7
prediction of number 8 is :6
prediction of number 9 is :2


In [683]:
#use Cross Entropy loss
feature = lambda x: x.flatten()
epochs=500
m = train_and_test(feature,epochs,model=model_auto_grad,lr=1,print_every=100)

epoch: 0, loss: 3.272478737144247
epoch: 100, loss: 3.098681159196158
epoch: 200, loss: 2.949812361865163
epoch: 300, loss: 2.7955675820042014
epoch: 400, loss: 2.5998120074725586
prediction of number 0 is :0
prediction of number 1 is :1
prediction of number 2 is :2
prediction of number 3 is :3
prediction of number 4 is :4
prediction of number 5 is :5
prediction of number 6 is :6
prediction of number 7 is :7
prediction of number 8 is :8
prediction of number 9 is :9


In [684]:
#use Cross Entropy Loss
feature = get_feature
epochs=500
m = train_and_test(feature,epochs,model=model_auto_grad,lr=0.01,print_every=100)

epoch: 0, loss: 3.3816390151883344
epoch: 100, loss: 3.2357709068355027
epoch: 200, loss: 3.17739580136705
epoch: 300, loss: 3.121404563687707
epoch: 400, loss: 3.0513159660598648
prediction of number 0 is :0
prediction of number 1 is :1
prediction of number 2 is :9
prediction of number 3 is :3
prediction of number 4 is :4
prediction of number 5 is :5
prediction of number 6 is :6
prediction of number 7 is :7
prediction of number 8 is :8
prediction of number 9 is :2


# Answers

In [685]:
def get_feature(x):
    feature=[0,0,0,0]
    def get_shadow(x,dim):
        feature  =torch.sum(x,dim)
        feature = feature.float()
        feature = feature.view(1,6)
        return feature
    feature  = get_shadow(x,0)
    return feature

In [686]:
def model(feature,weights):
    y=-1
    #[feature 1], added dimension to represent 'b' parameter
    feature = torch.cat((feature,torch.tensor(1.0).view(1,1)),1)
    feature2=feature.mul(feature)
    y = feature.mm(weights[:,0:1])+feature2.mm(weights[:,1:2])
    return y

In [699]:
def train_model(image_data,image_label,weights,lr):
    loss_value_before=100000000.
    loss_value=1000000.
    for epoch in range(0,3000):
        loss_value_before=loss_value
        loss_value=0
        for i in range(0,10):

            feature = get_feature(image_data[i])
            y = model(feature,weights)
            
            #L2 loss
            loss = 0.5*(y-image_label[i])*(y-image_label[i])

            loss_value += loss.data.item()

            # w  = w - (y-y1)*x*lr

            loss.backward()
            #weights.data.sub_(weights.grad.data*lr)
            #weights.grad.data.zero_()
            with torch.no_grad():
                weights -= weights.grad*lr
                weights.grad.zero_()
            #loss.data=
        if epoch%300 == 0:
            print("epoch=%s,loss=%s/%s,weights=%s"%(epoch,loss_value,loss_value_before,weights))
        #epoch+=1
        #loss_value=0
        #:loss=0
        #import pdb
        #pdb.set_trace()
    return weights

In [700]:

weights = torch.randn(7,2,requires_grad = True)
image_label = labels
image_data= generate_data()

print(image_data[0])
print("-"*20)


print(image_data[8])
print("-"*20)


lr = -0.05

weights=train_model(image_data,image_label,weights,lr)



for i in range(0,10):
    x=image_data[i]

    feature=get_feature(x)

    y = model(feature,weights)

    print(i,y,feature)

tensor([[0, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0]])
--------------------
tensor([[0, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0]])
--------------------
epoch=0,loss=3.928820914648576e+20/1000000.0,weights=tensor([[-4.1741e-01,  3.3697e-01],
        [-3.0446e+09, -6.0821e+09],
        [-3.1632e+09, -6.6751e+09],
        [-3.1955e+09, -6.8365e+09],
        [-7.2425e+09, -3.5432e+10],
        [-7.6787e+03, -7.6799e+03],
        [-1.5271e+09, -1.5271e+09]], requires_grad=True)
epoch=300,loss=nan/nan,weights=tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], requires_grad=True)
epoch=600,loss=nan/nan,weights=tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [na